In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.style.use('lenk')

### Read activities file and get all 2021 runs

In [3]:
df_activities = pd.read_csv('export/activities.csv')
df_activities = df_activities[['Activity ID', 'Activity Date', 'Filename', 'Activity Type']]
df_activities = df_activities.loc[df_activities['Filename'].notna()]

In [4]:
df_activities = df_activities.loc[df_activities['Activity Type']=='Run']

In [5]:
df_activities['Activity Date'] = pd.to_datetime(df_activities['Activity Date'])
df_activities = df_activities.loc[(df_activities['Activity Date']>='2021-01-01')&
                                  (df_activities['Activity Date']<='2021-12-31')]

In [6]:
df_activities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 35 to 230
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Activity ID    185 non-null    int64         
 1   Activity Date  185 non-null    datetime64[ns]
 2   Filename       185 non-null    object        
 3   Activity Type  185 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 7.2+ KB


In [7]:
activities_ids = df_activities['Activity ID'].values

### Read coordinates from .gpx files

In [8]:
import gpxpy

In [9]:
def get_gpx(activity_id):
    
    gpx_path = f'export/activities/{activity_id}.gpx'
    with open(gpx_path) as f:
        gpx = gpxpy.parse(f)

    points = []
    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'id':activity_id,
                'time': p.time,
                'latitude': p.latitude,
                'longitude': p.longitude,
            })

    df = pd.DataFrame.from_records(points)
    df['time_since'] = ((df['time'] - df['time'].min()) / np.timedelta64(1, 's')).astype(int)
    return df

In [10]:
df_list = []

for activity_id in activities_ids:
    df_list.append(get_gpx(activity_id))

gpx = pd.concat(df_list, ignore_index=True)

In [11]:
# excluding some runs
gpx = gpx.loc[gpx['longitude']>0]

In [12]:
gpx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388504 entries, 0 to 388503
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype                        
---  ------      --------------   -----                        
 0   id          388504 non-null  int64                        
 1   time        388504 non-null  datetime64[ns, SimpleTZ("Z")]
 2   latitude    388504 non-null  float64                      
 3   longitude   388504 non-null  float64                      
 4   time_since  388504 non-null  int64                        
dtypes: datetime64[ns, SimpleTZ("Z")](1), float64(2), int64(2)
memory usage: 17.8 MB


### Draw and save the map images

In [13]:
image_folder = 'images'

In [14]:
def draw_and_save_map(time_since):
    plt.figure(figsize=(10, 10))

    sns.scatterplot(x='longitude', 
                    y='latitude', 
                    data=gpx.loc[gpx['time_since']<=time_since], 
                    linewidth=0,
                    s=5,
                    alpha=.1)
    sns.scatterplot(x='longitude', 
                    y='latitude', 
                    data=gpx.loc[gpx['time_since']==time_since],
                    s=50,
                    color='red')

    plt.xlim(gpx['longitude'].min()-0.001, gpx['longitude'].max()+0.001)
    plt.ylim(gpx['latitude'].min()-0.001, gpx['latitude'].max()+0.001)
    plt.axis('off')

    minutes = time_since // 60
    seconds = time_since % 60

    plt.annotate(f'{minutes:02d}:{seconds:02d}', (gpx['longitude'].min()+0.002, gpx['latitude'].max()-0.002), size=20)

    plt.savefig(f'{image_folder}/{time_since}.png', dpi=100, bbox_inches='tight');
    plt.close()

In [15]:
r = gpx['time_since'].max()

for i in range(r):
    draw_and_save_map(i)

### Create the video

In [16]:
import cv2
import os

In [17]:
fps = 100

images = [str(i)+'.png' for i in range(r)]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter('video.avi', 0, fps, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()